In [1]:
import numpy as np

#import pandad used for data frames to import and store the data set
import pandas as pd

#import re used for splitting up the time ordered into day month year
import re

#used to display pd dataframes
from IPython.display import display

#Used for plotting Graphs
import matplotlib.pyplot as plt

In [2]:
def convert_timestamp(data):
    
    #Create new empty column in data set to store new timestamp object.
    if "Time Stamp" not in list(data.columns):
        data["Time Stamp"] = np.nan


    #Convert the string in the "Ordered At" column into a Pandas datestamp object and store
    # in new column

    for index, timestamp in data["Ordered At"].iteritems():
    
        #timestamp objects are in the form of US dates so convert the string into US date by switching the day and month
        timestampTemp = timestamp.split("/")
        timestampTemp1 = str(timestampTemp[1] + "/" + timestampTemp[0] + "/" + timestampTemp[2])
    
        #convert new string into timestamp object and store in new time stamp column
        data.set_value(index,"Time Stamp",pd.Timestamp(timestampTemp1))
    
    return data

In [3]:
def add_total_price(data):
    
    #Create new empty column in data set to store new timestamp object.
    if "Total Price" not in list(data.columns):
        data["Total Price"] = np.nan


    for index, price in data["Price"].iteritems():
        
        temp_total_price = price * (data["Quantity"][index])
    

        data.set_value(index,"Total Price",temp_total_price)
    
    return data

In [4]:
def add_next_order_date_days (data, restaurant, product):
    """
    Function that takes a data set and returns the same data set but with a new column giving the next order date next to 
    the product number given.
    
    """
    if "Next Order Days" not in list(data.columns):
        data["Next Order Days"] = np.nan
    
    tempData = data[(data["Restaurant"] == restaurant) & (data["Product"]==product)]
    
    for index, timestamp in tempData["Time Stamp"].iteritems():
        timestamp_list = []
        next_order_found = False
    
        for index2, timestamp2 in tempData["Time Stamp"].iteritems():
            if timestamp2 > timestamp:
                timestamp_list.append(timestamp2)
                next_order_found = True
    
        if next_order_found:
            next_date = min(timestamp_list)
            days = (next_date - timestamp).days
            data.set_value(index,"Next Order Days",days)  
            data.set_value(index,"Next Order Date",next_date)

    return data

In [5]:
def import_data(data):
    #import data to pandas data frame
    
    #creat list of all the restaurants used to loop functions.
    resturaunt_set = set(data["Restaurant"])
    
    #convert ordered at to timestamp
    data = convert_timestamp(data)
    data = add_total_price(data)
    #looping per resturaunt
    for resturaunt in resturaunt_set:
        data = add_next_order_date_days (data,resturaunt,38)
    
    return (data , resturaunt_set)

In [6]:
def dto_mean_predict(data,restaurant,product, last_order = None):
    """
    
    """
    temp_data = data[:][(data["Restaurant"]==restaurant)&(data["Product"]==product)]
    days_predict = temp_data["Next Order Days"].mean()
    if last_order == None:
        last_order = temp_data["Time Stamp"][temp_data["Next Order Date"].isnull()].all()
        #print(type(last_order))
    
    order_date_predict = last_order + pd.Timedelta(days_predict,'d')
    
    return order_date_predict

In [7]:
def dto_median_predict(data,restaurant,product, last_order = None):
    """
    
    """
    temp_data = data[:][(data["Restaurant"]==restaurant)&(data["Product"]==product)]
    days_predict = temp_data["Next Order Days"].median()
    if last_order ==None:
        last_order = temp_data["Time Stamp"][temp_data["Next Order Date"].isnull()].all()
    
    #print(type(last_order))
    order_date_predict = last_order + pd.Timedelta(days_predict,'d')
        
    return order_date_predict

In [8]:
def resturaunt_orders_count(data, product):
    resturaunt_set = set(data["Restaurant"])
    orders_count = pd.DataFrame(columns = ("Restaurant","Total Orders"))
    i = 0
    for temp_restaurant in resturaunt_set:
        count = data["Product"][data["Restaurant"]==temp_restaurant].count()
        temp_frame = pd.DataFrame(data = [[temp_restaurant,count]],columns=["Restaurant","Total Orders"], index = [i])
        orders_count = orders_count.append(temp_frame)
        i += 1
        
    return orders_count

In [9]:
def predict_test (start_date,train,data, restaurant, product = 38):
    
    """
    Function which outputs a table showing order date predictions against the actual
    order date, to be used to see if the prediction models are accurate. 
    
    """
    
    columns = ['Prediction 1','Prediction 2','actual order date','difference 1','difference 2']
    testlist = pd.DataFrame(columns=columns)
    index = 0
        
    while type(start_date) == pd.tslib.Timestamp :
        p1 = dto_mean_predict(train,restaurant,product, last_order = start_date)
        p2 = dto_median_predict(train,restaurant,product, last_order = start_date)
        actual_next_order = find_next_order_date(data,start_date,restaurant,product)
        
        if type(actual_next_order) == pd.tslib.Timestamp:
            difference1 = (p1 - actual_next_order).days
            difference2 = (p2 - actual_next_order).days
            frame_data = {
            'Prediction 1':p1,
            'Prediction 2':p2,
            'actual order date':actual_next_order,
            'difference 1':difference1,
            'difference 2':difference2            
            }
            tempframe = pd.DataFrame(data=frame_data, columns = columns, index=[index])
            testlist = testlist.append(tempframe)
            index +=1
        
        start_date = actual_next_order
        
    return testlist 

In [75]:
def find_lateness(summary_data,restaurant,product, current_date, last_order):
    """
    
    """
    
    days_predict = summary_data["Median Order Days"][(summary_data["Restaurant"]==restaurant)&(summary_data["Product"]==product)].values[0]
    print(days_predict)
    
    try:
        order_date_predict = last_order + pd.Timedelta(days_predict,'d')
        lateness_score = (current_date - order_date_predict).days / days_predict
        lateness_score = (lateness_score * 
                          summary_data["Median Total Price"][(summary_data["Restaurant"]==restaurant)&(summary_data["Product"]==product)])
    
    #a negative lateness means that the order is not late as teh predicted order date is ahead of the current date
    
    #a positive lateness means that the order is late and should be expected soon. 
    #The score is mulitplied by the median total order value to give it a monetary worth.
    
    except:
        lateness_score = np.nan
    
    
    return lateness_score

In [52]:
def lateness_table(current_date, product, data,summary_data,last_order=None):
    resturaunt_set = summary_data["Restaurant"]
    
    
    j = 0
    lateness = pd.DataFrame(columns=["Restaurant","Product","Lateness"])
    for r in resturaunt_set:
        
        if last_order==None:
            last_order = max(data["Time Stamp"][(data["Restaurant"]==r)&(data["Product"]==product)
                                            & (data["Time Stamp"]<= date)])
            
            print(last_order)
        
        temp_lateness = find_lateness(summary_data,r,product,current_date,last_order)
        
        
        frame_data = {
            "Restaurant":r,
            "Product":product,
            "Lateness":temp_lateness
        }
                   
        temp_frame = pd.DataFrame(data=frame_data,index=[j],columns=["Restaurant","Product","Lateness"])
        lateness = lateness.append(temp_frame)
        j+=1
    return lateness

In [26]:
def restaurant_summary(data):
    i = 0
    columns= ["Restaurant","Product", "Median Price", "Median Quantity","Median Total Price","Median Order Days","Total Orders"]
    summary_table = pd.DataFrame(columns=columns)
    
    resturaunt_set = set(data["Restaurant"])
    product_set = set(data["Product"])
    
    for r in resturaunt_set:
        for p in product_set:
            
            m_price = data["Price"][(data["Restaurant"]==r)&(data["Product"]==p)].median()
            m_days = data["Next Order Days"][(data["Restaurant"]==r)&(data["Product"]==p)].median()
            m_quantity = data["Quantity"][(data["Restaurant"]==r)&(data["Product"]==p)].median()
            o_count = data["Product"][(data["Restaurant"]==r)&(data["Product"]==p)].count()
            m_tprice = data["Total Price"][(data["Restaurant"]==r)&(data["Product"]==p)].median()
            
           
            frame_data = {
                    "Restaurant":r,
                    "Product":p,
                    "Median Price": m_price,
                    "Median Quantity":m_quantity,
                    "Median Total Price": m_tprice,
                    "Median Order Days": m_days,
                    "Total Orders": o_count
                }
            temp_frame = pd.DataFrame(data=frame_data,columns=columns,index = [i])
            
            summary_table = summary_table.append(temp_frame)
            i+=1
            
     

    return summary_table

In [19]:
read = pd.read_csv('data1.csv')

In [20]:
all_data, r_set = import_data(read)

In [28]:
all_data_summary = restaurant_summary(all_data)
all_data.v

In [74]:
all_data_summary["Median Order Days"][(all_data_summary["Restaurant"]==28)&(all_data_summary["Product"]==38)].values[0]

3.0

In [42]:
test_date = pd.Timestamp(year=2017,month=1,day=1)
last_order = pd.Timestamp(year=2016,month=12,day=14)

In [76]:
lateness_table(test_date,38,all_data,all_data_summary,last_order)

4.0
nan
11.0
5.0
nan
nan
11.0
19.5
5.0
nan
4.0
5.0
3.0
1.0
nan
6.0
nan
20.0
5.0
12.0
nan
nan
5.0
24.0
2.0
3.0
8.5
30.0
nan
7.0
4.0
2.0
6.0
4.0
nan
13.0
7.0
34.0
27.5
9.0
nan
7.0
nan
6.0
24.0
2.5
nan
14.0
3.0
4.0
nan
nan
7.0
nan
13.0
nan
5.5
nan
10.5
6.0
nan
18.5
nan
4.0
23.0
14.0
20.0
6.5
nan
nan
4.5
nan
nan
4.0
61.5
4.0
6.0
8.0
27.5
25.0
8.0
nan
nan
14.5
nan
7.0
16.5
5.0
75.0
nan
3.0
16.5
5.0
18.0
nan
69.0
16.0
28.0
22.0
nan
6.0
7.0
9.0
12.0
nan
nan
6.5
nan
nan
4.0
34.0
9.0
5.0
5.0
3.0
7.0
nan
7.0
nan
8.0
14.0
31.0
nan
4.0
nan
20.5
2.0
6.0
4.0
62.5
9.5
26.0
nan
25.0


,Restaurant,Product,Lateness
0,516.0,38.0,154.000000
1,517.0,38.0,NaN
2,7.0,38.0,25.136364
3,8.0,38.0,117.000000
4,11.0,38.0,NaN
5,524.0,38.0,NaN
6,18.0,38.0,51.545455
7,21.0,38.0,-5.005128
8,23.0,38.0,88.400000
9,25.0,38.0,NaN
